In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.preprocessing import LabelEncoder

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input/a490e594-6-dataset/Dataset"))

# Any results you write to the current directory are saved as output.

In [ ]:
train1 = pd.read_csv('../input/a490e594-6-dataset/Dataset/train.csv')
test = pd.read_csv('../input/a490e594-6-dataset/Dataset/test.csv')
build_owner = pd.read_csv('../input/a490e594-6-dataset/Dataset/Building_Ownership_Use.csv')
build_struct = pd.read_csv('../input/a490e594-6-dataset/Dataset/Building_Structure.csv')
sample = pd.read_csv('../input/a490e594-6-dataset/Dataset/sample_submission.csv')


In [ ]:
train_b = train1.merge(build_owner , on=['building_id','district_id','vdcmun_id'])
train_b_s = train_b.merge(build_struct , on=['building_id','district_id','vdcmun_id','ward_id'])
train_b_s = train_b_s.reset_index(drop=True)


In [ ]:
test_b = test.merge(build_owner , on=['building_id','district_id','vdcmun_id'])
test_b_s = test_b.merge(build_struct , on=['building_id','district_id','vdcmun_id','ward_id'])
test_b_s = test_b_s.reset_index(drop=True)

In [ ]:
flooors_damaged =train_b_s['count_floors_pre_eq']-train_b_s['count_floors_post_eq']
train_b_s['floors_damaged'] = flooors_damaged

In [ ]:
flooors_damaged1=test_b_s['count_floors_pre_eq']-test_b_s['count_floors_post_eq']
test_b_s['floors_damaged'] = flooors_damaged1

In [ ]:
train_b_s = train_b_s.drop(['count_floors_pre_eq','count_floors_post_eq'], axis=1)
test_b_s = test_b_s.drop(['count_floors_pre_eq','count_floors_post_eq'], axis=1)

In [ ]:
train_b_s.loc[train_b_s.age_building>=90, 'age_building']=90
test_b_s.loc[test_b_s.age_building>=90, 'age_building']=90

In [ ]:
df = pd.DataFrame()
df1 = pd.DataFrame()

In [ ]:
train_b_s.head()

In [ ]:
df['period 1'] = train_b_s['has_superstructure_rc_engineered']*0.075*train_b_s['height_ft_pre_eq'].pow(0.75)
df1['period 1'] = test_b_s['has_superstructure_rc_engineered']*0.075*test_b_s['height_ft_pre_eq'].pow(0.75)

In [ ]:
rc_trans = train_b_s['has_superstructure_rc_engineered'].replace({0:1, 1:0})
rc_trans1 = test_b_s['has_superstructure_rc_engineered'].replace({0:1, 1:0})

In [ ]:
df['period2'] = rc_trans * ((0.09 * train_b_s['height_ft_pre_eq'])/(train_b_s['plinth_area_sq_ft'].pow(0.5)))
df1['period2'] = rc_trans1 * ((0.09 * test_b_s['height_ft_pre_eq'])/(test_b_s['plinth_area_sq_ft'].pow(0.5)))

In [ ]:
df['period'] = df.sum(axis=1)
df1['period'] = df.sum(axis=1)

In [ ]:
train_b_s['period'] = df['period']
test_b_s['period'] = df1['period']

In [ ]:
train_b_s = train_b_s.drop(['height_ft_pre_eq','height_ft_post_eq'], axis=1)
test_b_s = test_b_s.drop(['height_ft_pre_eq','height_ft_post_eq'], axis=1)

In [ ]:
train_b_s['has_repair_started']=train_b_s['has_repair_started'].fillna(-1)
train_b_s['count_families'] = train_b_s['count_families'].fillna(1.0)

In [ ]:
test_b_s['has_repair_started']=test_b_s['has_repair_started'].fillna(-1)

In [ ]:
bid = test_b_s['building_id']

In [ ]:
y = train_b_s['damage_grade']

In [ ]:

for c in ['district_id','vdcmun_id','ward_id']:
    lbl = LabelEncoder()
    lbl.fit(list(train_b_s[c].unique())+list(test_b_s[c].unique()))
    train_b_s[c] = lbl.transform(train_b_s[c])
    test_b_s[c] = lbl.transform(test_b_s[c])
    print(c)

In [ ]:
train_b_s.head()

In [ ]:
test_b_s = test_b_s.drop(['building_id'], axis=1)

In [ ]:
train_b_s = train_b_s.drop(['building_id','damage_grade'], axis=1)



In [ ]:
print(train_b_s.shape)
print(test_b_s.shape)

In [ ]:
train_b_s = pd.get_dummies(train_b_s)
test_b_s = pd.get_dummies(test_b_s)

In [ ]:
print(train_b_s.shape)
print(test_b_s.shape)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train_b_s, y, test_size= 0.2,  random_state=0)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(max_features=52,random_state=0, n_estimators = 130,min_samples_split=8, min_samples_leaf=4 , max_depth = 24 ,n_jobs =-1 , criterion='entropy')

rf.fit(X_train,y_train)


In [ ]:
# from sklearn.model_selection import cross_val_score
# accuracies = cross_val_score(estimator = rf, X = train_b_s, y = y, cv = 3)

In [ ]:
rf_pred = rf.predict(X_test)

In [ ]:
rf_pred1 = rf.predict(test_b_s)

In [ ]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test , rf_pred))

In [ ]:
result = pd.DataFrame()
result['building_id'] = bid
result['damage_grade']= rf_pred1
result.head()

In [ ]:
result.to_csv('zoutput2.csv', index=False)